# Downloading ECCO Data

Author: Mike Wood

This notebook is designed to download ECCO data from the [ECCO Drive](https://ecco.jpl.nasa.gov/drive/).

Note that you must have a [NASA Earthdata](https://www.earthdata.nasa.gov/) account. If you don't already have one, be sure to make one before using this notebook.

When you first navigate to the [ECCO Drive](https://ecco.jpl.nasa.gov/drive/), you will be directed to a page with your WebDAV/Programmatic API credentials - these will be used when prompted for your credentials below.

#### Import the modules for this notebook

In [2]:
import requests
from requests.auth import HTTPBasicAuth
import os
import calendar

#### Define the destination file path

In [3]:
# define the path to the folder where the data will be downloaded
download_folder = '/Users/johnathonlu/Documents/SJSU/cs185c_spring2025/Data/ECCO'

#### Define the data request

In [4]:
# choose an ECCO version and release
version = 'Version5' #e.g. Version4
release = 'Alpha' #e.g. Release4

# choose a subset
subset = 'latlon_daily' #e.g. interp_monthly

# choose the variable to download
var_name = 'SST' # e.g. SSHDYN

# choose a start year and end year (in range 1992-2017)
start_year = 2015
end_year = 2015

#### Make a list of file paths
Nothing to do here - just run this cell to generate the list.

In [5]:
# define the base url
base_url = 'https://ecco.jpl.nasa.gov/drive/files/'+version+'/'+release+'/'+subset+'/'+var_name+'.nc'

# make a list of URls corresponding to the file dates
file_urls = []
for year in range(start_year,end_year+1):
    for month in range(1,13):
        # find days in a month (ranges from 29-31)
        num_days = calendar.monthrange(year, month)[1]
        for day in range(1, num_days + 1):
            yr = str(year)
            mo = '{:02d}'.format(month)
            dy = '{:02d}'.format(day)
            file_name = var_name + '_' + yr + mo + dy + '.nc'
            file_url = base_url + '/' + file_name
            file_urls.append(file_url)


#### Download the data

In [6]:
print('Enter your WebDAV/Programmatic API credentials')
# different from NASA login
username = input('Enter your username: kuetone')
password = input('Enter your password: MSz1xgpj3xFRu#6OtJk')

# check that all of the subdirectories are created
if version not in os.listdir(download_folder):
    os.mkdir(os.path.join(download_folder,version))
if release not in os.listdir(os.path.join(download_folder,version)):
    os.mkdir(os.path.join(download_folder,version,release))
if subset not in os.listdir(os.path.join(download_folder,version,release)):
    os.mkdir(os.path.join(download_folder,version,release,subset))
if var_name not in os.listdir(os.path.join(download_folder,version,release,subset)):
    os.mkdir(os.path.join(download_folder,version,release,subset,var_name))

# download all of the data
for i in range(len(file_urls)):
    url = file_urls[i]
    file_name = url.split('/')[-1]
    if file_name not in os.listdir(os.path.join(download_folder,version,release,subset,var_name)):
        print('Downloading '+file_name+' (file '+str(i+1)+' of '+str(len(file_urls))+')')
        output_file = os.path.join(download_folder,version,release,subset,var_name,file_name)
        with requests.get(url, stream=True, allow_redirects=True, auth=HTTPBasicAuth(username, password)) as r:
            r.raise_for_status()
            with open(output_file, 'wb') as f:                   
                for chunk in r.iter_content(chunk_size=8192):   
                    f.write(chunk)

Enter your WebDAV/Programmatic API credentials


Enter your username: kuetone kuetone
Enter your password: MSz1xgpj3xFRu#6OtJk MSz1xgpj3xFRu#6OtJk


Note: if the above cell fails before the download is complete, try re-running - often the API will reject downloads when there are too many pings in a given time interval.